# Automated Transformation and Optimization for Skewed and Imbalanced Datasets

## Introduction
This notebook integrates **data preprocessing, model optimization, and performance evaluation** to improve machine learning results on skewed and imbalanced datasets.
**The pipeline is designed to be flexible and can be applied to multiple datasets.**


In [584]:
import seaborn as sns
import lightgbm as lgb
import time
from IPython.display import display
from sklearn.metrics import classification_report
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import confusion_matrix, precision_recall_curve
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import boxcox
from sklearn.preprocessing import PowerTransformer
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

## Integrating the Previous Project into the Current Research
The previous project successfully demonstrated how class balancing and feature transformations impact model performance. These same principles are applied in the current research, but with a broader focus on automating preprocessing techniques for diverse datasets.

- **Class Balancing** (SMOTE & Undersampling): In the wine quality dataset, balancing improved model accuracy by ensuring all classes were well represented. Similarly, in the current research, we apply controlled oversampling and undersampling to create a more robust classification pipeline.

- **Skewness Correction**: The previous project applied log transformations to normalize distributions, preventing biases. Here, we expand this with Box-Cox and Yeo-Johnson transformations, adapting preprocessing to different data types.

- **Model Optimization**: In the last project, hyperparameter tuning boosted performance dramatically. We extend this by optimizing classification thresholds and cost-sensitive learning to handle imbalanced datasets more effectively.

In [585]:
# # Load dataset
# data = pd.read_csv("winequality-white.csv", sep=';')
#
# # Visualize initial class distribution
# plt.figure(figsize=(8, 5))
# sns.countplot(x='quality', data=data)
# plt.title('Wine Quality Distribution - Before Balancing', fontsize=14)
# plt.xlabel('Quality', fontsize=12)
# plt.ylabel('Frequency', fontsize=12)
# plt.show()
#
# # Split features and target
# X = data.drop(columns=['quality'])
# y = data['quality']
#
# # Display initial class distribution
# print("Class Distribution Before Balancing:", Counter(y))
#
# # Step 1: Handle Extremely Rare Classes (Manual Duplication)
# min_samples = 30  # Minimum samples required for SMOTE to work
#
# for cls, count in Counter(y).items():
#     if count < min_samples:
#         extra_samples = data[data['quality'] == cls].sample(min_samples - count, replace=True)
#         data = pd.concat([data, extra_samples])
#
# # Update X and y after rare class handling
# X = data.drop(columns=['quality'])
# y = data['quality']
#
# # Step 2: Apply SMOTE to Underrepresented Classes
# max_class_count = max(Counter(y).values())
# smote_strategy = {cls: max(count, int(0.6 * max_class_count)) for cls, count in Counter(y).items() if count < 0.6 * max_class_count}
#
# smote = SMOTE(sampling_strategy=smote_strategy, random_state=42)
# X_smote, y_smote = smote.fit_resample(X, y)
#
# # Display new class distribution after SMOTE
# print("Class Distribution After SMOTE:", Counter(y_smote))
#
# # Visualize class distribution after SMOTE
# plt.figure(figsize=(8, 5))
# sns.countplot(x=y_smote)
# plt.title('Wine Quality Distribution - After SMOTE', fontsize=14)
# plt.xlabel('Quality', fontsize=12)
# plt.ylabel('Frequency', fontsize=12)
# plt.show()
#
# # Step 3: Apply Proper Undersampling for Overrepresented Classes
# min_class_count = min(Counter(y_smote).values())  # Smallest class after SMOTE
# undersample_strategy = {}
#
# for cls, count in Counter(y_smote).items():
#     if count > min_class_count * 1.5:  # Reduce dominant classes but keep them slightly higher than others
#         undersample_strategy[cls] = int(min_class_count * 1.5)  # Keep majority at 1.5x the smallest class
#
# # Apply undersampling
# undersample = RandomUnderSampler(sampling_strategy=undersample_strategy, random_state=42)
# X_final, y_final = undersample.fit_resample(X_smote, y_smote)
#
# # Display final class distribution after proper balancing
# print("Final Class Distribution After SMOTE & Corrected Undersampling:", Counter(y_final))
#
# # Visualizing target distribution after full balancing
# plt.figure(figsize=(8, 5))
# sns.countplot(x=y_final)
# plt.title('Wine Quality Distribution - After SMOTE & Corrected Undersampling', fontsize=14)
# plt.xlabel('Quality', fontsize=12)
# plt.ylabel('Frequency', fontsize=12)
# plt.show()
#


### How This Improves Prediction?

In [586]:
# # Load the baseline model
# baseline_model = joblib.load('baseline_model.pkl')
#
# # Use the final balanced dataset
# X = X_final
# y = y_final
#
# # Split dataset into train and test
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#
# # Predict on the test set using the baseline model
# y_pred_baseline = baseline_model.predict(X_test)
#
# # Calculate baseline performance metrics
# r2_baseline = r2_score(y_test, y_pred_baseline)
# mse_baseline = mean_squared_error(y_test, y_pred_baseline)
# rmse_baseline = np.sqrt(mse_baseline)
#
# print(f"Baseline Model Performance:\nR^2 Score: {r2_baseline:.4f}\nMSE: {mse_baseline:.4f}\nRMSE: {rmse_baseline:.4f}")
#
# # Hyperparameter tuning with GridSearchCV for XGBoost
# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'max_depth': [3, 5, 7],
#     'learning_rate': [0.01, 0.1, 0.2],
#     'subsample': [0.8, 1.0]
# }
#
# grid_search = GridSearchCV(
#     estimator=xgb.XGBRegressor(),
#     param_grid=param_grid,
#     scoring='neg_mean_squared_error',
#     cv=5,
#     verbose=1
# )
#
# # Fit GridSearch on the balanced dataset
# grid_search.fit(X_train, y_train)
# best_model = grid_search.best_estimator_
# print(f'Best Parameters: {grid_search.best_params_}')
#
# # Evaluate the improved model
# y_pred = best_model.predict(X_test)
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# rmse = np.sqrt(mse)
#
# print(f'Improved Model Performance:\nRMSE: {rmse:.4f}, R^2: {r2:.4f}')
#
# # Create a comparison table
# comparison_df = pd.DataFrame({
#     'Metric': ['R^2', 'MSE', 'RMSE'],
#     'Baseline Model': [r2_baseline, mse_baseline, rmse_baseline],
#     'Improved Model': [r2, mse, rmse]
# })
#
# print("\nComparison of Baseline and Improved Models:")
# print(comparison_df)
#
# # Plot comparison of R^2
# plt.figure(figsize=(8, 5))
# bar_width = 0.35
# indices = np.arange(len(comparison_df))
#
# plt.bar(indices, comparison_df['Baseline Model'], bar_width, label='Baseline Model', color='blue')
# plt.bar(indices + bar_width, comparison_df['Improved Model'], bar_width, label='Improved Model', color='green')
#
# plt.xlabel('Metrics')
# plt.ylabel('Scores')
# plt.title('Comparison of Model Performance')
# plt.xticks(indices + bar_width / 2, comparison_df['Metric'])
# plt.legend()
# plt.show()


The improved model shows a significant enhancement in predictive performance compared to the baseline model. The R² score increased from 0.769 to 0.928, indicating that the new model explains much more variance in the data. The MSE dropped from 0.85 to 0.26, demonstrating a major reduction in prediction errors. Additionally, the RMSE decreased from 0.92 to 0.51, confirming that the model's predictions are more precise and stable.

##### **These improvements highlight the impact of data balancing and hyperparameter tuning on predictive accuracy. By applying SMOTE & undersampling, the model now performs better across all quality levels, rather than favoring the dominant ones.**

### Broader Impact: Scalable Preprocessing for Various Datasets
While the previous project focused on a specific dataset (wine quality), the current research aims to generalize preprocessing techniques to work across multiple data types and domains. By automating class balancing and skew correction, we develop a versatile, adaptable pipeline that enhances machine learning models regardless of dataset characteristics. 

In [587]:
!pip install lightgbm

You should consider upgrading via the 'C:\Users\guyha\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


## Step 0: Preprocessing the Dataset
**Loads the dataset from the specified file path**

    - Displays dataset structure and basic statistics.

In [588]:
import pandas as pd

def load_data(filepath,target_column):
    """
    Loads the credit card fraud dataset, extracts numerical features, and returns
    the DataFrame along with a list of numerical columns (excluding the target column).

    Parameters:
        filepath (str): Path to the CSV file.

    Returns:
        df (pd.DataFrame): Loaded dataset.
        numerical_features (list): List of numerical feature column names (excluding the target).
    """
    # Load dataset
    df = pd.read_csv(filepath)

    # Define target column


    # Extract numerical features excluding the target column
    numerical_features = df.select_dtypes(include=['number']).columns.tolist()
    if target_column in numerical_features:
        numerical_features.remove(target_column)

    # Display dataset shape and first few rows
    print(f"Dataset Shape: {df.shape}")
    print("\nSample Rows:")
    display(df.head())

    return df, numerical_features  # Return DataFrame and numerical feature names

target_column = 'Class'
df, numerical_features = load_data("creditcard.csv",target_column)


Dataset Shape: (284807, 31)

Sample Rows:


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [589]:
df_dtypes = pd.DataFrame(df.dtypes).reset_index()
df_dtypes.columns = ['Column', 'Data Type']
print(df_dtypes)

    Column Data Type
0     Time   float64
1       V1   float64
2       V2   float64
3       V3   float64
4       V4   float64
5       V5   float64
6       V6   float64
7       V7   float64
8       V8   float64
9       V9   float64
10     V10   float64
11     V11   float64
12     V12   float64
13     V13   float64
14     V14   float64
15     V15   float64
16     V16   float64
17     V17   float64
18     V18   float64
19     V19   float64
20     V20   float64
21     V21   float64
22     V22   float64
23     V23   float64
24     V24   float64
25     V25   float64
26     V26   float64
27     V27   float64
28     V28   float64
29  Amount   float64
30   Class     int64


## Step 1: Detecting and Correcting Skewed Features

In [590]:
import pandas as pd

def detect_skewed_features(df, numerical_features, skew_threshold=0.5):
    """
    Detects numerical features with skewness above a given threshold.

    Parameters:
        df (pd.DataFrame): The input dataset.
        numerical_features (list): List of numerical feature column names.
        skew_threshold (float): The threshold above which features are considered skewed.

    Returns:
        skewed_features (list): List of skewed numerical feature names.
        skewness_table (pd.DataFrame): DataFrame with skewness values for all numerical features.
    """
    print("Detecting skewed features...")

    # Compute skewness for numerical features
    skewness_values = df[numerical_features].skew()

    # Select features where absolute skewness is greater than the threshold
    skewed_features = skewness_values[abs(skewness_values) > skew_threshold].index.tolist()

    # Store skewness values in a DataFrame for reference
    skewness_table = pd.DataFrame({
        "Feature": numerical_features,
        "Skewness": skewness_values
    }).sort_values(by="Skewness", ascending=False)

    print(f"Skewed Features (>|{skew_threshold}|): {skewed_features}")

    return skewed_features, skewness_table  # Return both the feature names and skewness values

# Example usage:
skewed_features, skewness_table = detect_skewed_features(df, numerical_features)


Detecting skewed features...
Skewed Features (>|0.5|): ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V12', 'V14', 'V16', 'V17', 'V20', 'V21', 'V23', 'V24', 'V26', 'V27', 'V28', 'Amount']


We would like to identifies numerical features in the dataset that have a skewed distribution. Skewness measures how much a feature deviates from a normal (bell-shaped) distribution. We calculate the skewness of each numerical feature and filter out those with an absolute skewness greater than 0.5, which indicates moderate to high skewness. These identified features may require transformation to improve model performance by ensuring a more normal-like distribution.

### **1.1 Addressing the Issue of Worsened Skewness After Transformation**

In some cases, applying transformations actually increased the skewness rather than reducing it.
To fix this, we will implement a smarter transformation selection approach that avoids making skewness worse.

Three Improvements to Prevent Skewness Worsening:
1. Add a Post-Transformation Check:
If the skewness increases instead of decreasing, we will revert to the original data.
2. Choose a More Intelligent Transformation:
If all values are strictly positive, we apply Box-Cox.
If the feature contains negative values, we use Yeo-Johnson instead.
We avoid log transformations automatically, as they may worsen skewness in extreme cases.
3. Store the Original Skewness and Compare:
After applying a transformation, we check if skewness improved.
If the transformation made it worse, we undo the change and restore the original values.


In [591]:
def best_transform(feature_series):
    """
    Try several transformations and select the one that yields the lowest absolute skewness.
    Returns the transformed series, the name of the chosen method, the original skew, and the new skew.
    """
    original_skew = feature_series.skew()
    best_method = 'none'
    best_skew = abs(original_skew)
    best_transformed = feature_series.copy()

    # Dictionary to store transformation results
    transforms = {}

    # Try Box-Cox (only if all values are positive)
    if (feature_series > 0).all():
        try:
            transformed, _ = boxcox(feature_series + 1)
            skew_val = pd.Series(transformed).skew()
            transforms['boxcox'] = (transformed, abs(skew_val))
        except Exception as e:
            pass

    # Try Log1p transform (with shift)
    try:
        transformed = np.log1p(feature_series - feature_series.min() + 1)
        skew_val = pd.Series(transformed).skew()
        transforms['log1p'] = (transformed, abs(skew_val))
    except Exception as e:
        pass

    # Try Yeo-Johnson
    try:
        pt = PowerTransformer(method='yeo-johnson')
        transformed = pt.fit_transform(feature_series.values.reshape(-1, 1)).flatten()
        skew_val = pd.Series(transformed).skew()
        transforms['yeo-johnson'] = (transformed, abs(skew_val))
    except Exception as e:
        pass

    # Try QuantileTransformer (maps data to a normal distribution)
    try:
        qt = QuantileTransformer(output_distribution='normal', random_state=0)
        transformed = qt.fit_transform(feature_series.values.reshape(-1, 1)).flatten()
        skew_val = pd.Series(transformed).skew()
        transforms['quantile'] = (transformed, abs(skew_val))
    except Exception as e:
        pass

    # Compare all transformations and select the best one if it improves skewness
    for method, (trans, skew_val) in transforms.items():
        if skew_val < best_skew:
            best_skew = skew_val
            best_method = method
            best_transformed = trans

    return best_transformed, best_method, original_skew, pd.Series(best_transformed).skew()

### **The following transformations are applied based on methodologies discussed in the article "Skewness Be Gone: Transformative Tricks for Data Scientists" by Vinod Chugani.**
   #### **Quantile Transformation**
   - **How it works:**
     - Maps the feature values to a specified distribution, such as normal, based on their quantiles.
     - Effectively addresses skewness by distributing data points evenly across the chosen distribution.
   - **Best for:** Severe skewness, non-Gaussian distributions, and handling outliers.

   #### **Yeo-Johnson Transformation**
   - **How it works:**
     - A power transformation adaptable to both positive and non-positive data.
     - Applies an optimal transformation parameter to stabilize variance and reduce skewness.
   - **Best for:** Moderate skewness, features with zero or negative values, and when data includes negative or zero values.

   #### **Log1p (Natural Log + 1) Transformation**
   - **How it works:**
     - Uses a logarithmic transformation to compress large values and reduce right-skewness.
     - Applies the natural logarithm to each data point after adding one, effectively handling zero and positive values.
   - **Best for:** Mild skewness, right-skewed data, and features where small values dominate.

   #### Applying Data Transformations

   To address skewness in our data, we applied several transformations as recommended in the article ["Skewness Be Gone: Transformative Tricks for Data Scientists" by Vinod Chugani](https://machinelearningmastery.com/skewness-be-gone-transformative-tricks-for-data-scientists/):

   - **Log1p Transformation:** Suitable for mildly skewed, right-skewed data, especially when small values dominate.
   - **Yeo-Johnson Transformation:** Ideal for moderate skewness and data that includes zero or negative values.
   - **Quantile Transformation:** Effective for severe skewness and non-Gaussian distributions, as it maps data to a normal distribution.

In [592]:
def transform_and_store_skewed_features(df, skewed_features):
    """
    Transforms skewed features and stores results without modifying df inside the function.
    Returns a list of transformed data and skewness values.
    """
    results = []
    transformed_data = []  # Store transformation details for later plotting

    for feature in skewed_features:
        skew_before = df[feature].skew()

        # Get the best transformation
        transformed, method, orig_skew, new_skew = best_transform(df[feature])

        # Store results in a list instead of modifying df directly
        transformed_data.append({
            "feature": feature,
            "original": df[feature].copy(),  # Store original data before transformation
            "transformed": transformed,  # Store transformed data
            "skew_before": skew_before,
            "skew_after": new_skew,
            "method": method
        })

        results.append([feature, orig_skew, new_skew, method])

        # Apply transformation to df (only if needed immediately)
        df[feature] = transformed

    # Convert results into a DataFrame
    results_df = pd.DataFrame(results, columns=["Feature", "Skewness Before", "Skewness After", "Method"])

    return transformed_data, results_df


# ---------------------------
# Detect skewed features
# ---------------------------
skew_threshold = 0.5
numerical_skew = df[numerical_features].skew()
skewed_features = numerical_skew[abs(numerical_skew) > skew_threshold].index.tolist()
print("Skewed Features:", skewed_features)

# ---------------------------
# Step 1: Transform Data and Get Results
# ---------------------------
transformed_data, results_df = transform_and_store_skewed_features(df, skewed_features)

# ---------------------------
# Step 2: Display the Table FIRST
# ---------------------------
# display(results_df)
# # ---------------------------
# # Step 3: Plot Histograms for Each Transformed Feature
# # ---------------------------
# for data in transformed_data:
#     feature = data["feature"]
#     skew_before = data["skew_before"]
#     new_skew = data["skew_after"]
#
#     fig, axes = plt.subplots(1, 2, figsize=(16, 6))  # 1 Row, 2 Columns
#
#     # Histogram Before Transformation
#     sns.histplot(data["original"], bins=100, kde=True, ax=axes[0], color='blue', alpha=0.6)
#     axes[0].set_title(f'Before Transformation: {feature} (Skew: {skew_before:.2f})')
#
#     # Histogram After Transformation
#     sns.histplot(data["transformed"], bins=100, kde=True, ax=axes[1], color='red', alpha=0.6)
#     axes[1].set_title(f'After Transformation: {feature} (Skew: {new_skew:.2f})')
#
#     plt.tight_layout()  # Adjust layout to prevent overlap
#     plt.show()


Skewed Features: ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V12', 'V14', 'V16', 'V17', 'V20', 'V21', 'V23', 'V24', 'V26', 'V27', 'V28', 'Amount']


In this table, we see the impact of our **automated transformation process** on the **skewness of numerical features**. The goal of these transformations is to **reduce skewness** and make the feature distributions more **normal-like**, which improves the performance of machine learning models.

#### **Observations & Improvements**
- **Most features had a high initial skewness**, either negative or positive.
- **The process successfully reduced skewness for all features**, making them more normally distributed.
- **Different transformation methods were applied**:
  - **Quantile Transformation** was the most commonly chosen method, as it is effective for severe skewness.
  - **Yeo-Johnson Transformation** was applied for `Amount` and `V4`, as it worked better in reducing skewness for these features.

#### **Why Does This Matter?**
- **Reducing skewness** helps **stabilize variance** and improves the **effectiveness of machine learning models**.
- **Highly skewed features** can lead to models being **biased** toward extreme values.
- By using **appropriate transformations**, we ensure that features contribute **meaningfully** to model predictions.

In [593]:
# # ---------------------------
# # Step 3: Plot Only the Last Transformed Feature with Boxplots
# # ---------------------------
# if transformed_data:  # Ensure there's at least one feature
#     last_feature_data = transformed_data[-1]  # Get last transformed feature
#
#     feature = last_feature_data["feature"]
#     skew_before = last_feature_data["skew_before"]
#     new_skew = last_feature_data["skew_after"]
#
#     fig, axes = plt.subplots(2, 2, figsize=(20, 12))  # 2 Rows, 2 Columns
#
#     # Histogram Before Transformation
#     sns.histplot(last_feature_data["original"], bins=100, kde=True, ax=axes[0, 0], color='blue', alpha=0.6)
#     axes[0, 0].set_title(f'Before Transformation: {feature} (Skew: {skew_before:.2f})')
#
#     # Boxplot Before Transformation
#     sns.boxplot(x=last_feature_data["original"], ax=axes[0, 1], color='cyan')
#     axes[0, 1].set_title(f'Boxplot Before: {feature}')
#
#     # Histogram After Transformation
#     sns.histplot(last_feature_data["transformed"], bins=100, kde=True, ax=axes[1, 0], color='red', alpha=0.6)
#     axes[1, 0].set_title(f'After Transformation: {feature} (Skew: {new_skew:.2f})')
#
#     # Boxplot After Transformation
#     sns.boxplot(x=last_feature_data["transformed"], ax=axes[1, 1], color='orange')
#     axes[1, 1].set_title(f'Boxplot After: {feature}')
#
#     plt.tight_layout()  # Adjust layout to prevent overlap
#     plt.show()


### 1.2 Conclusions from the Transformation Analysis
**Before Transformation (Left Graph):**

The feature "Amount" had a high skewness (16.98), meaning the distribution was heavily skewed to the right.
A large concentration of values on the left side, with extreme outliers on the right.

**Boxplot (Middle Graph):**

Most values are clustered near the lower end of the range.
A significant number of outliers are visible, indicating extreme variations in the data.

**After Transformation (Right Graph):**

Skewness significantly reduced (0.32), bringing the distribution closer to normal.
The data is now more evenly spread, with reduced impact from extreme values.

### 1.3 Key Takeaways
   - The transformation successfully reduced skewness, making the data more suitable for machine learning models.
   - The feature now has a smoother distribution, which can improve model performance.
   - Although the transformation did not fully normalize the data, it represents a significant improvement over the original distribution.

## **Step 2: Addressing Class Imbalance in Machine Learning**  

Class imbalance is a common challenge in datasets where one class is significantly underrepresented compared to another. This issue is especially critical in **fraud detection, medical diagnostics, and rare event classification**, where models tend to favor the majority class, leading to poor detection of minority cases.

To tackle this, we implement a **balanced approach** that enhances model performance while preserving real-world data characteristics:

✔ **TF-IDF Transformation:** Converts textual data into numerical features, ensuring text-based patterns contribute effectively to classification.  

✔ **Moderate SMOTE Oversampling:** Expands the minority class to **10% of the majority class size**, improving its representation without overfitting.  

✔ **Strategic Undersampling:** Reduces the majority class to **twice the size of the minority class**, but only in the **training set**, as recommended by research for optimal generalization.  

✔ **Maintaining Real-World Distribution in the Test Set:** The test set remains **unaltered** to reflect actual class proportions, ensuring the model is evaluated under realistic conditions.  

By combining **oversampling, undersampling, and TF-IDF transformation**, we create a **well-balanced training dataset** that allows the model to learn from **minority cases** without introducing bias towards artificially inflated data. 

### 2.1 Apply TF-IDF for Text Processing
**Term Frequency-Inverse Document Frequency (TF-IDF)** is a statistical measure employed to evaluate the importance of a word within a document relative to a collection of documents (corpus). It is widely used in natural language processing (NLP) and information retrieval to transform textual data into numerical representations, facilitating the analysis and modeling of text by machine learning algorithms.

**Components of TF-IDF:**

1. **Term Frequency (TF):** This component quantifies the frequency of a term in a specific document. It reflects how often a word appears in a document, indicating its relevance within that particular context.

2. **Inverse Document Frequency (IDF):** This component assesses the significance of a term across the entire corpus. It diminishes the weight of terms that occur frequently across many documents and increases the weight of terms that are rare, thereby highlighting words that are more informative and less generic.

**Calculation of TF-IDF:**

The TF-IDF value for a term in a document is calculated by multiplying its TF and IDF values:

**_TF-IDF(t, d, D) = TF(t, d) x IDF(t, D)_** 

Where:
- t represents the term;
- d denotes the specific document;
- D signifies the entire corpus of documents.

**Applications of TF-IDF:**

TF-IDF is instrumental in various NLP tasks, including:

- **Text Classification:** By converting text into numerical features, TF-IDF enables machine learning models to classify documents based on their content.

- **Information Retrieval:** Search engines utilize TF-IDF to rank documents by relevance to a user's query, ensuring more pertinent results are presented.

- **Clustering and Topic Modeling:** TF-IDF aids in grouping similar documents and identifying underlying themes by emphasizing significant terms within the text.

In the context of class imbalance, particularly in domains like fraud detection or rare event classification, TF-IDF plays a crucial role. By assigning appropriate weights to terms, it ensures that minority class characteristics are adequately represented, thereby enhancing the model's ability to detect and learn from underrepresented instances.

#### References to the Article
**"Data oversampling and imbalanced datasets: an investigation of performance for machine learning and feature engineering"**


In [594]:
from sklearn.feature_extraction.text import TfidfVectorizer
def apply_tfidf(data, text_column):
    """
    Applies TF-IDF transformation to a given text column in a DataFrame.
    Replaces the original text column with the extracted TF-IDF features.

    :param data: DataFrame containing the dataset
    :param text_column: Name of the column containing text data
    :return: Updated DataFrame with TF-IDF features
    """
    print(f"\n🔹 Applying TF-IDF on column: **{text_column}**")

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(data[text_column])

    # Convert sparse matrix to DataFrame
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(),
                            columns=vectorizer.get_feature_names_out())

    print(f"Extracted {len(tfidf_df.columns)} TF-IDF features for column: {text_column}")

    # Drop the original text column and concatenate TF-IDF features
    data = data.drop(columns=[text_column]).reset_index(drop=True)
    data = pd.concat([data, tfidf_df], axis=1)

    print(f"New dataset shape after TF-IDF transformation: {data.shape}")

    # Show a preview of the new TF-IDF features
    print("\nPreview of TF-IDF features:")
    print(tfidf_df.head(3))

    return data

In [596]:

# Detect all text columns dynamically
text_columns = df.select_dtypes(include=['object', 'string']).columns.tolist()

if text_columns:
    print(f"\n📊 Detected text columns for TF-IDF transformation: {text_columns}")
else:
    print("\n✅ No text columns detected. Skipping TF-IDF transformation.")

# Apply TF-IDF transformation if text columns exist
for col in text_columns:
    df = apply_tfidf(df, col)


✅ No text columns detected. Skipping TF-IDF transformation.


**What Happens Here?**

Automatically finds text columns → No need to manually specify them.

Applies TF-IDF transformation → Converts words into numerical values.

Ensures compatibility with SMOTE → Prevents errors when handling imbalanced data.

### 2.2 Balancing The dataset initially had a highly imbalanced class distribution:

In [ ]:
# Feature and target split
X = df.drop(columns=[target_column])  # Features
y = df[target_column]  # Target variable

# Display class distribution before balancing
print("Class Distribution Before Balancing:", Counter(y))

Class Distribution Before Balancing:

    - Class 0 (No Fraud): 284,315 samples
    - Class 1 (Fraud): Only 492 samples!
This is extremely imbalanced!

### 2.1.1. Applying SMOTE and its Variants for Imbalanced Data
#### Data oversampling and imbalanced datasets: an investigation of performance for machine learning and feature engineering Muhammad Mujahid et al. (2024)
#### Overview
Class imbalance is a critical issue in machine learning that can lead to biased models favoring the majority class. Oversampling techniques, particularly **Synthetic Minority Oversampling Technique (SMOTE)** and its variants, help address this issue by generating synthetic samples for the minority class. This notebook explores the application of **SMOTE, Borderline-SMOTE, SVM-SMOTE, and ADASYN** for handling imbalanced datasets.

#### Why SMOTE?
According to the article **"Data oversampling and imbalanced datasets: an investigation of performance for machine learning and feature engineering"** by Mujahid et al. (2024), oversampling techniques significantly improve model performance when dealing with class-imbalanced datasets. The study found that:

- **SMOTE and ADASYN achieved higher accuracy** compared to other oversampling techniques.
- **Support Vector Machine (SVM) with a linear kernel** achieved the highest accuracy (99.67%) when using **ADASYN oversampled datasets with TF-IDF features**.
- **Borderline-SMOTE improved classification along decision boundaries**, preventing models from learning misleading patterns from majority classes.
- **SVM-SMOTE uses support vectors to generate synthetic samples**, making it effective for datasets where decision boundaries are complex.

#### References to the Article
- **Synthetic Minority Oversampling Technique (SMOTE):** Introduced by Chawla et al. and used to generate synthetic samples by interpolating between existing minority class samples. ([Section: Description of oversampling techniques])
- **Borderline-SMOTE:** Enhances SMOTE by focusing on samples near the decision boundary, preventing overfitting. ([Section: Border-line SMOTE])
- **SVM-SMOTE:** Uses support vectors instead of KNN to generate synthetic samples, making it useful for complex boundaries. ([Section: SVM-SMOTE])
- **ADASYN:** Generates synthetic data adaptively based on data density, improving learning for underrepresented minority class samples. ([Section: Adaptive synthetic oversampling])

---

### 2.1.2. SMOTE Implementation in Python
The function below applies different SMOTE variations to balance an imbalanced dataset.

In [ ]:
import pandas as pd
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, SVMSMOTE, ADASYN
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

def apply_smote(X, y, smote_type='SMOTE', sampling_strategy=0.1, random_state=42):
    """
    Apply different SMOTE variations to balance an imbalanced dataset.

    Parameters:
    - X: Feature matrix
    - y: Target labels
    - smote_type: Type of SMOTE ('SMOTE', 'Borderline', 'SVM', 'ADASYN')
    - sampling_strategy: Ratio of minority to majority class
    - random_state: Random seed for reproducibility

    Returns:
    - X_resampled: Resampled feature matrix
    - y_resampled: Resampled target labels
    """
    smote_mapping = {
        'SMOTE': SMOTE(random_state=random_state),
        'Borderline': BorderlineSMOTE(random_state=random_state),
        'SVM': SVMSMOTE(random_state=random_state),
        'ADASYN': ADASYN(random_state=random_state),
    }

    if smote_type not in smote_mapping:
        raise ValueError("Invalid smote_type. Choose from 'SMOTE', 'Borderline', 'SVM', 'ADASYN'.")

    smote = smote_mapping[smote_type]
    class_counts = Counter(y)
    num_classes = len(class_counts)

    print(f"\nClass Distribution Before {smote_type}: {class_counts}")

    # If binary classification, apply the original behavior
    if num_classes == 2:
        smote.set_params(sampling_strategy=sampling_strategy)

    # If multi-class, adjust sampling strategy
    else:
        print(f"\n🔹 Multi-class detected ({num_classes} classes). Adjusting strategy.")
        majority_class = max(class_counts, key=class_counts.get)

        # Create a dictionary for multi-class balancing
        sampling_strategy_dict = {
            class_label: max(int(class_counts[majority_class] * sampling_strategy), count)
            for class_label, count in class_counts.items() if count > 5 and class_label != majority_class
        }

        if sampling_strategy_dict:
            smote.set_params(sampling_strategy=sampling_strategy_dict)
            print(f"🔹 **Adjusted Sampling Strategy:** {sampling_strategy_dict}")
        else:
            print("⚠️ Not enough samples for ADASYN. Switching to SMOTE instead.")
            smote = SMOTE(sampling_strategy='auto', random_state=random_state)

    # Apply SMOTE/ADASYN and catch errors
    try:
        X_resampled, y_resampled = smote.fit_resample(X, y)
    except ValueError as e:
        print(f"\n⚠️ {smote_type} Error: {e}")
        print("👉 Switching to standard SMOTE instead.")
        smote = SMOTE(sampling_strategy=sampling_strategy, random_state=random_state)
        X_resampled, y_resampled = smote.fit_resample(X, y)

    class_counts_after = Counter(y_resampled)
    print(f"\nClass Distribution After {smote_type}: {class_counts_after}")

    # Visualization of class distribution
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    sns.barplot(x=list(class_counts.keys()), y=list(class_counts.values()), ax=axes[0])
    axes[0].set_title(f"Before {smote_type}")
    axes[0].set_ylabel("Count")

    sns.barplot(x=list(class_counts_after.keys()), y=list(class_counts_after.values()), ax=axes[1])
    axes[1].set_title(f"After {smote_type}")
    axes[1].set_ylabel("Count")

    plt.show()

    return X_resampled, y_resampled


# Example usage:
X_smote, y_smote = apply_smote(X, y, smote_type='ADASYN')



### Explanation of the Process
- The function **supports different SMOTE variations**: `SMOTE`, `Borderline-SMOTE`, `SVM-SMOTE`, and `ADASYN`.
- It prints **class distribution before and after oversampling**.
- **Visualizes the class distribution** to show the effect of oversampling.

#### Observations Based on the Article
- **SMOTE and ADASYN provide the best performance** on imbalanced Twitter datasets.
- **SVM with linear kernel showed the highest accuracy (99.67%) on ADASYN-generated data**.
- **TF-IDF features performed better** than BoW for text classification.
- **Borderline-SMOTE is useful when minority samples are close to decision boundaries**.

---

### Conclusion
- **Why Use This?** The choice of oversampling strategy can impact model performance, as seen in the study. Selecting **ADASYN or SMOTE** typically leads to **higher accuracy**.
- **Recommended Approach:** If working with text data, use **TF-IDF** for feature extraction before applying SMOTE.
- **Best Performing Model:** **SVM with ADASYN** oversampling achieved **99.67% accuracy**, making it the most reliable choice.

Class Distribution After SMOTE Balancing:

    - Class 0 (No Fraud): 284,315 samples
    - Class 1 (Fraud): 28526 synthetic samples

#### 2.1.2. Apply undersampling
 Majority class will be twice the minority class size

In [ ]:
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

def apply_undersampling(X, y, sampling_ratio=0.5, random_state=42, max_classes_display=10):
    """
    Apply Random Undersampling to balance a dataset.

    Parameters:
    - X: Feature matrix
    - y: Target labels
    - sampling_ratio: Ratio of minority to majority class for binary;
                      for multi-class, a dictionary will be generated.
    - random_state: Random seed for reproducibility.
    - max_classes_display: Maximum number of classes to display in the output.

    Returns:
    - X_balanced: Resampled feature matrix.
    - y_balanced: Resampled target labels.
    """
    class_counts = Counter(y)
    num_classes = len(class_counts)

    # 📊 **Print Class Distribution Before**
    print("\n📊 **Class Distribution Before Undersampling:**")

    if num_classes <= 2:  # Binary classification - print full details
        for label, count in class_counts.items():
            print(f"- Class {label}: {count} samples")
    else:  # Multi-class - Show only the most frequent classes
        print(f"🔹 Multi-class detected ({num_classes} unique classes). Showing top {max_classes_display}:")
        most_common_classes = class_counts.most_common(max_classes_display)
        for label, count in most_common_classes:
            print(f"- Class {label}: {count} samples")
        print(f"... ({num_classes - max_classes_display} more classes not shown)")

    # Multi-class: Adjust strategy
    if num_classes > 2:
        min_class_size = min(class_counts.values())  # Find the smallest class size
        if min_class_size <= 1:
            print("\n⚠️ Warning: Some classes have only 1 sample. Adjusting undersampling strategy.")
            sampling_strategy_dict = {
                class_label: min(class_counts[class_label], max(1, int(class_counts[class_label] * sampling_ratio)))
                for class_label in class_counts
            }
        else:
            sampling_strategy_dict = {
                class_label: max(1, int(class_counts[class_label] * sampling_ratio))
                for class_label in class_counts
            }

        undersample = RandomUnderSampler(sampling_strategy=sampling_strategy_dict, random_state=random_state)

    else:  # Binary classification
        undersample = RandomUnderSampler(sampling_strategy=sampling_ratio, random_state=random_state)

    try:
        X_balanced, y_balanced = undersample.fit_resample(X, y)
        class_counts_after = Counter(y_balanced)

        # 📊 **Print Class Distribution After**
        print("\n✅ **Class Distribution After Undersampling:**")
        if num_classes <= 2:  # Binary case
            for label, count in class_counts_after.items():
                print(f"- Class {label}: {count} samples")
        else:  # Multi-class case, show summary
            print(f"🔹 {num_classes} classes after undersampling. Showing top {max_classes_display}:")
            most_common_after = class_counts_after.most_common(max_classes_display)
            for label, count in most_common_after:
                print(f"- Class {label}: {count} samples")
            print(f"... ({num_classes - max_classes_display} more classes not shown)")

    except ValueError as e:
        print(f"\n⚠️ Error during undersampling: {e}")
        print("👉 Falling back to equal class distribution.")
        undersample = RandomUnderSampler(sampling_strategy="auto", random_state=random_state)
        X_balanced, y_balanced = undersample.fit_resample(X, y)
        print(f"✅ Successfully applied equal class distribution.")

    return X_balanced, y_balanced



# Apply undersampling to the dataset

X_balanced, y_balanced = apply_undersampling(X_smote, y_smote)


If left unbalanced, a model would likely predict "No Fraud" for almost everything and achieve high accuracy but poor fraud detection.

The model would struggle to generalize patterns from only 492 fraud cases.

Class Distribution After Undersampling:

    - Class 0 (No Fraud): 57052 samples
    - Class 1 (Fraud): 28526 synthetic samples


**Visualizing Class Distribution Before and After Balancing**

In [ ]:
# Plot class distribution before and after balancing
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.countplot(x=y, ax=axes[0])
axes[0].set_title("Class Distribution Before Balancing")
axes[0].set_xlabel("Class")
axes[0].set_ylabel("Count")

sns.countplot(x=y_balanced, ax=axes[1])
axes[1].set_title("Class Distribution After SMOTE + Undersampling")
axes[1].set_xlabel("Class")
axes[1].set_ylabel("Count")

plt.tight_layout()
plt.show()

In [ ]:
import time
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve
from sklearn.model_selection import train_test_split

def train_evaluate_lgbm(X, y, X_balanced, y_balanced, test_size=0.2, train_size=0.3, random_state=42):
    # Split the imbalanced dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, train_size=train_size, random_state=random_state, stratify=y)

    # Split the balanced dataset
    X_train_bal, X_test_bal, y_train_bal, y_test_bal = train_test_split(
        X_balanced, y_balanced, test_size=test_size, train_size=train_size, random_state=random_state, stratify=y_balanced
    )

    # Define LightGBM classifier parameters
    lgb_params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'learning_rate': 0.05,
        'num_leaves': 31,
        'max_depth': -1,
        'random_state': random_state
    }

    # Train LightGBM on imbalanced dataset
    start_time = time.time()
    model_lgb = lgb.LGBMClassifier(**lgb_params)
    model_lgb.fit(X_train, y_train)
    end_time = time.time()

    # Predictions on imbalanced test data
    y_pred_lgb = model_lgb.predict(X_test)
    y_probs_lgb = model_lgb.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

    # Evaluate performance on imbalanced dataset
    print(f"Performance on Imbalanced Dataset (Training Time: {end_time - start_time:.2f} sec):")
    print(classification_report(y_test, y_pred_lgb))

    # Train LightGBM on balanced dataset
    start_time = time.time()
    model_lgb_bal = lgb.LGBMClassifier(**lgb_params)
    model_lgb_bal.fit(X_train_bal, y_train_bal)
    end_time = time.time()

    # Predictions on balanced test data
    y_pred_bal_lgb = model_lgb_bal.predict(X_test_bal)
    y_probs_bal_lgb = model_lgb_bal.predict_proba(X_test_bal)[:, 1]  # Probabilities for the positive class

    # Evaluate performance on balanced dataset
    print(f"Performance on Balanced Dataset (Training Time: {end_time - start_time:.2f} sec):")
    print(classification_report(y_test_bal, y_pred_bal_lgb))

    return y_test, y_pred_lgb, y_probs_lgb, y_test_bal, y_pred_bal_lgb, y_probs_bal_lgb

# Example call to the function
y_test, y_pred_lgb, y_probs_lgb, y_test_bal, y_pred_bal_lgb, y_probs_bal_lgb = train_evaluate_lgbm(X, y, X_balanced, y_balanced)

### **Explanation of LightGBM Results on Imbalanced vs. Balanced Datasets**

#### **1. Understanding the Context**
The output shows the results of **LightGBM (a gradient boosting model)** trained on both an **imbalanced dataset** and a **balanced dataset**. The dataset appears to be a **binary classification task** (fraud detection or another imbalanced scenario) with labels:
- **Class 0** (Negative class, majority)
- **Class 1** (Positive class, minority)

The model's performance is measured using **precision, recall, f1-score, and accuracy** on each dataset.

---

## **Imbalanced Dataset Training Results**
```
[LightGBM] [Info] Number of positive: 148, number of negative: 85294
```
- **148 fraud cases (positive class)**
- **85,294 non-fraud cases (negative class)**
- **Highly imbalanced (~0.17% fraud cases)**

### **Training Info**
```
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007051 seconds.
[LightGBM] [Info] Total Bins 7650
[LightGBM] [Info] Number of data points in the train set: 85442, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001732 -> initscore=-6.356647
[LightGBM] [Info] Start training from score -6.356647
```
- **Training on 85,442 samples** with **30 features**
- The prior probability of fraud cases (`pavg=0.001732`) is extremely low.
- The **initial model score** for LightGBM is **-6.356647**, meaning it's starting heavily biased toward the majority class.

### **Performance on Imbalanced Dataset**
```
Performance on Imbalanced Dataset (Training Time: 0.49 sec):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.50      0.65      0.57        98

    accuracy                           1.00     56962
   macro avg       0.75      0.83      0.78     56962
weighted avg       1.00      1.00      1.00     56962
```

### **Key Observations**
- **Accuracy = 1.00 (~100%)** → **Misleading due to class imbalance**
- **Class 0 (Majority, non-fraud)**: Precision, recall, and F1-score are all **1.00** (perfect) because almost all predictions are for Class 0.
- **Class 1 (Minority, fraud cases)**:
  - **Precision = 0.50** → Only 50% of predicted fraud cases were actually fraud.
  - **Recall = 0.65** → The model correctly detected 65% of actual fraud cases but still missed 35%.
  - **F1-score = 0.57** → Imbalanced performance; the model struggles with fraud detection.

#### **Problem with Imbalanced Data**
- The model is highly **biased toward the majority class**.
- Fraud cases are often **undetected or misclassified**.
- **Precision (0.50) is low**, meaning there are a lot of false positives.
- **Recall (0.65) is also not great**, meaning some fraud cases are missed.

---

## **Balanced Dataset Training Results**
```
[LightGBM] [Info] Number of positive: 8558, number of negative: 17115
```
- **8,558 fraud cases**
- **17,115 non-fraud cases**
- **Balanced dataset (1:2 ratio of fraud to non-fraud cases)**

### **Training Info**
```
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002783 seconds.
[LightGBM] [Info] Total Bins 7650
[LightGBM] [Info] Number of data points in the train set: 25673, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.333346 -> initscore=-0.693089
[LightGBM] [Info] Start training from score -0.693089
```
- **Training on 25,673 samples** (less than the imbalanced dataset, due to balancing techniques).
- The prior probability of fraud cases (`pavg=0.333346`) is **significantly higher** than in the imbalanced dataset.
- **Initial model score (-0.693089)** is closer to a fair classification (compared to -6.356647 before).

### **Performance on Balanced Dataset**
```
Performance on Balanced Dataset (Training Time: 0.26 sec):
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     11411
           1       1.00      0.99      0.99      5705

    accuracy                           0.99     17116
   macro avg       0.99      0.99      0.99     17116
weighted avg       0.99      0.99      0.99     17116
```

### **Key Observations**
- **Accuracy = 0.99 (~99%)** → Still high, but more reliable now.
- **Class 0 (Non-fraud cases)**:
  - **Precision = 0.99, Recall = 1.00, F1-score = 1.00** → Almost perfect.
- **Class 1 (Fraud cases)**:
  - **Precision = 1.00** → Every fraud case predicted was actually fraud.
  - **Recall = 0.99** → Almost every actual fraud case was detected.
  - **F1-score = 0.99** → Balanced model performance.

#### **Why Balanced Data Performs Better**
- The **recall for fraud cases increased from 0.65 to 0.99** → The model is now detecting **almost all** fraud cases.
- The **precision for fraud cases increased from 0.50 to 1.00** → Fewer false positives.
- The **model generalizes better** because it learns from an equal distribution of both classes.
- **False negative cases (missed fraud cases) are dramatically reduced**.

---

## **Final Comparison: Imbalanced vs. Balanced**
| Metric        | Imbalanced Dataset | Balanced Dataset |
|--------------|------------------|----------------|
| **Accuracy** | **1.00** (Misleading) | **0.99** (More reliable) |
| **Class 1 Precision (Fraud)** | **0.50** (50% of predicted fraud cases are correct) | **1.00** (No false positives) |
| **Class 1 Recall (Fraud)** | **0.65** (Misses 35% of fraud cases) | **0.99** (Almost all fraud cases detected) |
| **F1-score (Fraud)** | **0.57** | **0.99** |

---

## **Conclusion**
- The **imbalanced dataset caused poor fraud detection**, with **low recall (0.65) and precision (0.50)**.
- The **balanced dataset (via SMOTE or undersampling) improved model performance dramatically**.
- **Handling class imbalance is crucial for fraud detection models**; otherwise, models will be **biased toward the majority class**.
- **Recommendation:** Always **balance datasets** in classification problems where the minority class is underrepresented (e.g., fraud detection, rare disease classification).

**Confusion Matrices for Model Evaluation**

In [ ]:
def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No Fraud", "Fraud"], yticklabels=["No Fraud", "Fraud"])
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(title)
    plt.show()

# Confusion matrix for the imbalanced model
plot_confusion_matrix(y_test, y_pred_lgb, "Confusion Matrix - Imbalanced Dataset")

# Confusion matrix for the balanced model
plot_confusion_matrix(y_test_bal, y_pred_bal_lgb, "Confusion Matrix - Balanced Dataset")

### Confusion Matrix for the Imbalanced Dataset

| **Actual / Predicted** | **No Fraud (0)** | **Fraud (1)** |
|------------------------|-----------------|---------------|
| **No Fraud (0)**       | **56800** (TN)  | **64** (FP)  |
| **Fraud (1)**          | **34** (FN)     | **64** (TP)  |

- **TN (True Negative: 56800)** → Model **correctly** predicted non-fraud cases.
- **FP (False Positive: 64)** → Model **incorrectly** predicted fraud when it was actually non-fraud.
- **FN (False Negative: 34)** → Model **missed 34 fraud cases**, predicting them as non-fraud.
- **TP (True Positive: 64)** → Model **correctly** identified 64 fraud cases.

#### Observations*
**High Accuracy**: The model performs well overall because most transactions are **non-fraud**.
**Poor Fraud Detection**: Only **64 fraud cases** detected, missing **34 fraud transactions**.
**High Class Imbalance Impact**: The model is biased toward non-fraud cases due to the imbalanced dataset.



### Confusion Matrix for the Balanced Dataset

| **Actual / Predicted** | **No Fraud (0)** | **Fraud (1)** |
|------------------------|------------------|---------------|
| **No Fraud (0)**       | **11389** (TN)   | **22** (FP)   |
| **Fraud (1)**          | **83** (FN)       | **5622** (TP)   |

- **TN (True Negative: 11389)** → Model **correctly** predicted non-fraud cases.
- **FP (False Positive: 22)** → Model **incorrectly** predicted fraud when it was actually non-fraud.
- **FN (False Negative: 83)** → Model **missed 83 fraud cases**, predicting them as non-fraud.
- **TP (True Positive: 5622)** → Model **correctly** identified 5622 fraud cases.

#### Improvements After Balancing
**Much Better Fraud Detection**: **5622 fraud cases** correctly detected, compared to only **64** before.
**Lower False Positives**: The model only wrongly flagged **16** transactions as fraud.
**Increased Sensitivity to Fraud**: The model now **detects more fraud cases** at the cost of a few extra false positives.

### Key Takeaways
- The **imbalanced dataset** model **performs well in overall accuracy**, but **fails to detect fraud effectively**.
- The **balanced dataset** model **detects fraud cases much better**, reducing **false negatives** (missed fraud cases).
- **Balancing the dataset (e.g., SMOTE, undersampling)** **improves fraud detection**, making the model **more reliable**.


In [ ]:
def plot_precision_recall(y_true, y_probs, title):
    precision, recall, _ = precision_recall_curve(y_true, y_probs)
    plt.figure(figsize=(6, 5))
    plt.plot(recall, precision, marker='.')
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title(title)
    plt.show()

# Compare Precision-Recall Curves using returned probability predictions
plot_precision_recall(y_test, y_probs_lgb, "Precision-Recall Curve - Imbalanced Dataset")
plot_precision_recall(y_test_bal, y_probs_bal_lgb, "Precision-Recall Curve - Balanced Dataset")


### **Explanation of Precision-Recall (PR) Curves**
The **Precision-Recall (PR) Curve** helps evaluate the model's ability to **correctly detect fraud (positive class) while minimizing false positives**. It is particularly useful for **imbalanced datasets**, like our case of fraud detection.

The two graphs compare the **imbalanced dataset** (before applying SMOTE/undersampling) and the **balanced dataset** (after applying resampling techniques).

---

## **Precision-Recall Curve for the Imbalanced Dataset**
- The **curve starts at high precision (~1.0) but drops quickly** as recall increases.
- This means that as the model tries to detect more fraud cases (**higher recall**), it **starts making a lot of false positives**, causing **precision to drop**.
- The **rapid drop** suggests the model is **highly biased toward the majority class** (non-fraud), leading to **poor fraud detection**.
- **Key Takeaway:** The model **fails to maintain good precision as recall increases**, indicating **poor fraud detection performance**.

---

## **Precision-Recall Curve for the Balanced Dataset (Top Image)**
- The curve is **much more stable**, maintaining **high precision (~1.0) across most recall values**.
- This suggests that the **model now correctly detects fraud cases without sacrificing precision**.
- The **almost flat precision** at **high recall values** means the model **consistently identifies fraud cases correctly** with very few false positives.
- **Key Takeaway:** **Balancing the dataset (e.g., SMOTE or undersampling) significantly improves fraud detection** by ensuring the model does not ignore minority-class fraud cases.

---

## **Conclusion**
- **Imbalanced Dataset:** The model **struggles to detect fraud**. Precision drops quickly as it tries to detect more fraud cases.
- **Balanced Dataset:** The model **performs much better**, maintaining **high precision** even at higher recall.
- **Overall:** **Balancing the dataset helps improve fraud detection performance** by reducing false negatives (missed fraud) and keeping false positives under control.

## Step 3: Data Preprocessing and Model Training Pipeline
This stage focuses on automating data preprocessing and model training in a structured and reusable way.
The goal is to create a generalized pipeline that can be applied to any dataset with similar characteristics.


### **Step 1: Load the Dataset**
Since `winequality-white.csv` is different from `creditcard.csv`, we must load it while ensuring:
- **The target column is correctly identified** (likely `"quality"` instead of `"Class"`).
- **All numerical features are extracted properly**.

In [ ]:
# Load the new dataset
target_column = 'price'
df, numerical_features = load_data("USA_cars_datasets.csv",target_column)

### **Step 2: Detect Skewed Features**
The dataset contains numerical features (such as acidity, pH, alcohol, etc.). We check for **highly skewed features** using the function.

In [ ]:
# Detect skewed numerical features
skewed_features, skewness_table = detect_skewed_features(df, numerical_features)

# Display the skewness table
display(skewness_table)

### **Step 3: Transform Skewed Features**
To **normalize** the dataset and reduce skewness, apply the `best_transform()` function.

In [ ]:
# Transform and store skewed features
transformed_data, results_df = transform_and_store_skewed_features(df, skewed_features)

In [ ]:
# Display transformation results
display(results_df)

### **Step 4: Visualize the Before & After Skewness**
Since we previously plotted histograms, you can reuse the same approach.

In [ ]:
# Plot transformed distributions
for data in transformed_data:
    feature = data["feature"]
    skew_before = data["skew_before"]
    new_skew = data["skew_after"]

    fig, axes = plt.subplots(2, 2, figsize=(20, 12))  # 2 Rows, 2 Columns

    # Histogram Before Transformation
    sns.histplot(data["original"], bins=100, kde=True, ax=axes[0, 0], color='blue', alpha=0.6)
    axes[0, 0].set_title(f'Before Transformation: {feature} (Skew: {skew_before:.2f})')

    # Boxplot Before Transformation
    sns.boxplot(x=data["original"], ax=axes[0, 1], color='cyan')
    axes[0, 1].set_title(f'Boxplot Before: {feature}')

    # Histogram After Transformation
    sns.histplot(data["transformed"], bins=100, kde=True, ax=axes[1, 0], color='red', alpha=0.6)
    axes[1, 0].set_title(f'After Transformation: {feature} (Skew: {new_skew:.2f})')

    # Boxplot After Transformation
    sns.boxplot(x=data["transformed"], ax=axes[1, 1], color='orange')
    axes[1, 1].set_title(f'Boxplot After: {feature}')

    plt.tight_layout()  # Adjust layout
    plt.show()

In [ ]:

# Detect all text columns dynamically
text_columns = df.select_dtypes(include=['object', 'string']).columns.tolist()

if text_columns:
    print(f"\n📊 Detected text columns for TF-IDF transformation: {text_columns}")
    # Apply TF-IDF transformation to each text column
    for col in text_columns:
        df = apply_tfidf(df, col)
else:
    print("\n✅ No text columns detected. Skipping TF-IDF transformation.")

### **🚀 Step 5: Apply SMOTE or Undersampling (If Needed)**
Since **wine quality** is a **multi-class classification problem** (not binary fraud detection), SMOTE and undersampling **need adjustments**.

- If **some wine quality labels are underrepresented**, apply **SMOTE**.
- If **some labels are overrepresented**, apply **undersampling**.


In [ ]:
# Extract features (X) and target (y)
X = df[numerical_features]
y = df[target_column]

In [ ]:
# Apply SMOTE to balance classes
X_smote, y_smote = apply_smote(X, y, smote_type='ADASYN')

In [ ]:
# Apply undersampling if necessary
X_balanced, y_balanced = apply_undersampling(X_smote, y_smote)


### **🚀 Step 6: Apply TF-IDF (If There Are Text Columns)**
If the wine dataset contains text columns (unlikely, but possible for descriptions), you can **convert text features using TF-IDF**.


In [ ]:
df = apply_tfidf(df, text_column="description")  # Replace with actual text column name


## Step 3: Data Preprocessing and Model Training Pipeline
This stage focuses on automating data preprocessing and model training in a structured and reusable way.
The goal is to create a generalized pipeline that can be applied to any dataset with similar characteristics.

Steps in This Stage:
### 3.1 Splitting Dataset into Features and Target
First, we separate the dataset into the feature matrix X and target vector y. This function lets you specify the name of the target column, so it works for any dataset:


In [ ]:
def split_features_target(df, target_col):
    """
    Split a DataFrame into feature matrix X and target vector y.
    
    Parameters:
        df (pd.DataFrame): The input dataset.
        target_col (str): Name of the target column in df.
    Returns:
        X (pd.DataFrame): Features (all columns except target_col).
        y (pd.Series): Target labels.
    """
    X = df.drop(columns=[target_col])
    y = df[target_col].copy()
    return X, y


### 3.2 Balancing Classes (Oversampling & Undersampling)
To address class imbalance generically, we create a function that can oversample minority classes and undersample majority classes based on flexible criteria. By default, we’ll use SMOTE for oversampling and random undersampling for majority classes, but the thresholds are tunable:

In [ ]:
def balance_classes(X, y, oversample_target=0.6, undersample_factor=1.5, random_state=42):
    """
    Balance class distribution by oversampling minorities and undersampling majorities.
    
    Parameters:
        X (pd.DataFrame): Feature matrix.
        y (pd.Series or np.array): Target labels.
        oversample_target (float): Fraction of the largest class count to oversample minorities to.
                                   For example, 0.6 means upsample minority classes to ~60% of the max class size.
        undersample_factor (float): Maximum ratio of majority class size to minority class size after balancing.
                                    For example, 1.5 means no class will have more than 1.5x the number of samples of the smallest class.
        random_state (int): Random seed for reproducibility.
    Returns:
        X_bal (pd.DataFrame): Balanced feature matrix.
        y_bal (pd.Series): Balanced target labels.
    """
    # Count original class frequencies
    class_counts = Counter(y)
    max_class = max(class_counts, key=class_counts.get)      # class with largest count
    max_count = class_counts[max_class]
    
    # Define oversampling strategy: target counts for classes below the threshold
    oversample_strategy = {}
    for cls, count in class_counts.items():
        if count < oversample_target * max_count:
            # Set the new target count for this class (e.g., 0.6 * max_count)
            oversample_strategy[cls] = int(oversample_target * max_count)
    # Apply SMOTE oversampling for specified classes
    if oversample_strategy:
        smote = SMOTE(sampling_strategy=oversample_strategy, random_state=random_state)
        X_res, y_res = smote.fit_resample(X, y)
    else:
        X_res, y_res = X.copy(), y.copy()
    
    # After oversampling, get the smallest class count
    new_counts = Counter(y_res)
    min_count = min(new_counts.values())
    
    # Define undersampling strategy: limit classes that are more than undersample_factor * min_count
    undersample_strategy = {}
    for cls, count in new_counts.items():
        limit = int(undersample_factor * min_count)
        if count > limit:
            undersample_strategy[cls] = limit  # cap this class to the limit
    # Apply undersampling for specified classes
    if undersample_strategy:
        rus = RandomUnderSampler(sampling_strategy=undersample_strategy, random_state=random_state)
        X_bal, y_bal = rus.fit_resample(X_res, y_res)
    else:
        X_bal, y_bal = X_res, y_res
    
    return X_bal, y_bal

# Example usage on an imbalanced dataset:
X_bal, y_bal = balance_classes(X, y, oversample_target=0.6, undersample_factor=1.5)
print("Class counts after balancing:", Counter(y_bal))


How this works:
- We oversample any class that has fewer than oversample_target * (size of largest class). For instance, if the largest class has 1000 samples and oversample_target=0.6, then any class with <600 samples will be increased (via SMOTE) up to roughly 600. This parameter is flexible – setting it to 1.0 would fully balance to the majority class size, while a lower value avoids excessive synthetic data.
- After oversampling, we undersample any class that now has more than undersample_factor * (size of smallest class). For example, with undersample_factor=1.5, if the smallest class has 600 samples, no other class will be allowed more than 900; the majority class (if larger) is randomly down-sampled to 900. This prevents any class from still dominating the dataset.
- Both oversample_target and undersample_factor can be tuned per dataset. For datasets with extreme imbalance, you might oversample to 100% and undersample to equal counts; for milder cases, smaller adjustments may suffice.
- We use SMOTE and RandomUnderSampler from imblearn, but you could plug in different oversampling or undersampling methods if needed (the function could accept a different sampler object via parameters).

The result is a new balanced dataset (X_bal, y_bal) with a much more even class distribution, regardless of the original dataset’s class labels or imbalance severity.

### 3.3 Correcting Skewness in Numeric Features
This function will automatically try a few transformations and pick the one that best normalizes the data:

In [ ]:


def reduce_skewness(feature_series, methods=('boxcox', 'yeo-johnson', 'log'), plot=False):
    """
    Transform a numeric feature to reduce skewness.
    
    Tries multiple transformations and chooses the one with lowest absolute skew.
    Parameters:
        feature_series (pd.Series or np.array): The data for a single feature.
        methods (tuple of str): Transformation methods to try. Options: 'boxcox', 'yeo-johnson', 'log'.
        plot (bool): If True, plot before/after histograms (for analysis).
    Returns:
        transformed_series (pd.Series): Transformed feature data (same length as input).
        best_method (str): Name of the transformation chosen (or 'none' if no transform improved skew).
    """
    # Calculate original skewness
    original_skew = feature_series.skew()
    best_method = 'none'
    best_skew = abs(original_skew)
    best_transformed = feature_series.copy()
    
    # Try each requested transformation
    for method in methods:
        transformed = None
        try:
            if method == 'log':
                # Shift data to be positive (if needed) and apply log1p
                transformed = np.log1p(feature_series - feature_series.min() + 1)
            elif method == 'boxcox':
                # Box-Cox requires all positive values
                if (feature_series <= 0).any():
                    raise ValueError("Box-Cox requires positive values")
                transformed, _ = boxcox(feature_series)  # returns transformed data and lambda
            elif method == 'yeo-johnson':
                pt = PowerTransformer(method='yeo-johnson')
                # PowerTransformer returns a 2D array, so reshape is needed
                transformed = pt.fit_transform(feature_series.values.reshape(-1, 1)).flatten()
            else:
                continue  # unknown method
        except Exception as e:
            # Skip this method if it fails (e.g., boxcox on non-positive data)
            continue
        
        # Compute skew of the transformed data
        new_skew = pd.Series(transformed).skew()
        if abs(new_skew) < best_skew:
            best_skew = abs(new_skew)
            best_method = method
            best_transformed = pd.Series(transformed, index=feature_series.index)
    
    # (Optional) plot before vs. after if requested
    if plot:
        fig, axes = plt.subplots(1, 2, figsize=(10,4))
        axes[0].hist(feature_series, bins=30, color='skyblue')
        axes[0].set_title(f"Original Distribution (skew={original_skew:.2f})")
        axes[1].hist(best_transformed, bins=30, color='orange')
        axes[1].set_title(f"Transformed ({best_method}) (skew={best_skew:.2f})")
        plt.show()
    
    return best_transformed, best_method

# Example usage: fix skewness of 'Amount' feature
df['Amount_transformed'], method_used = reduce_skewness(df['Amount'], methods=('log','yeo-johnson','boxcox'))
print(f"Applied {method_used} transform to 'Amount'")


How this works:
- We check the feature’s current skew (original_skew). The function then tries each of the specified methods in turn:
    **Log transform (log)**: Uses log1p (log(1+x)) after shifting the data to be positive if needed. Great for right-skewed data with large outliers.
    **Box-Cox (boxcox)**: A power transform that only works if all values are positive. It finds an optimal exponent λ to normalize the distribution.
    **Yeo-Johnson (yeo-johnson)**: A power transform similar to Box-Cox but can handle zero or negative values as well.
- For each method that succeeds, we compute the absolute skew of the transformed data. The function selects the transformation that yields the lowest skew (i.e., most symmetric distribution). If none of the transforms improve the skew (which is rare), it will return the original data (best_method remains 'none').
- Flexible Parameters: You can choose which methods to try via the methods tuple. For example, on a dataset that has only positive values, you might include 'boxcox'; for a dataset with zeros/negatives, use 'yeo-johnson' or 'log'. You can also enable plot=True to visualize the effect for any feature.
T- his function handles any numeric feature series. In practice, you would apply it to each numeric column in your dataset that exhibits high skewness (for example, if abs(skew) > 1 as a rule of thumb). You might loop over all numeric features of X and replace those that are significantly skewed with their transformed version. For instance:

In [ ]:
for col in X.select_dtypes(include='number').columns:
    skew_val = X[col].skew()
    if abs(skew_val) > 1.0:  # if highly skewed
        X[col], method = reduce_skewness(X[col], methods=('log','yeo-johnson','boxcox'))
        print(f"Transformed {col} using {method} (new skew={X[col].skew():.2f})")


By using reduce_skewness on the **Amount** field in the credit card data, the extreme outliers (very large transactions) were tamed, making the model less biased toward those large values. This approach can be applied to any skewed feature in any dataset.

## Using the Generic Functions on a New Dataset
With these generic functions, you can construct a preprocessing pipeline for any new dataset. For example:


In [ ]:
from sklearn.datasets import load_breast_cancer


# פונקציה להדמיית התפלגות הקטגוריות
def plot_class_distribution(y, title):
    sns.countplot(x=y)
    plt.title(title)
    plt.xlabel('Class')
    plt.ylabel('Frequency')
    plt.show()

# טעינת הדאטהסט
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target  # שם עמודת המטרה הנכון

# הצגת התפלגות המחלקות לפני איזון
plot_class_distribution(df['target'], 'Class Distribution Before Balancing')

# שימוש בפונקציות הכלליות עם שם העמודה הנכון
X_new, y_new = split_features_target(df, target_col="target")

# איזון מחלקות
X_new_bal, y_new_bal = balance_classes(X_new, y_new, oversample_target=1.0, undersample_factor=1.0)

# תיקון סקיואנס לתכונות מספריות
for col in X_new_bal.select_dtypes(include='number').columns:
    if abs(X_new_bal[col].skew()) > 1.0:
        X_new_bal[col], method = reduce_skewness(X_new_bal[col], methods=('log','yeo-johnson','boxcox'))
        print(f"{col} transformed via {method}")
        


In the snippet above, we demonstrate a full pipeline: splitting data, balancing it, transforming skewed features, scaling, and encoding. Each step is **independent and reusable**. You can mix and match these functions or adjust their parameters as needed:
- If a dataset is not imbalanced, you can skip balance_classes or call it with parameters that effectively leave data unchanged.
- If features aren’t skewed, reduce_skewness will detect that (no transformation applied if it doesn’t improve skew).
- If all features are already numeric and scaled, you might not need scale_features or encode_categoricals.

- By converting the specific steps from the credit card dataset into these **flexible functions**, we ensure that the data preprocessing pipeline is not tied to any one dataset. You can apply this pipeline (or its parts) to any new dataset with minimal changes — just plug in the dataset and specify relevant columns or thresholds. This makes your data preparation process consistent, modular, and easily adaptable to new problems.

In [ ]:
#????????????!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# Load dataset
df = pd.read_csv("creditcard.csv")
target_column = 'Class'
numerical_features = df.select_dtypes(include=['number']).columns.tolist()
numerical_features.remove(target_column)
# Display dataset shape and first few rows
print(f"Dataset Shape: {df.shape}")
print("\nSample Rows:")
display(df.head())

# Set the figure size
fig, axes = plt.subplots(nrows=len(numerical_features)//3 + 1, ncols=3, figsize=(18, len(numerical_features)*1.5))

# Flatten axes array to iterate over
axes = axes.flatten()

# Plot histograms for all numerical features
for i, feature in enumerate(numerical_features):
    sns.histplot(df[feature], bins=100, kde=True, ax=axes[i], color='blue', alpha=0.6)
    axes[i].set_title(f'Distribution of {feature}')
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel("Count")

# Adjust layout for better spacing
plt.tight_layout()
plt.show()